In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
import os
if not os.path.exists('/content/train'):
    !gsutil -m cp -r gs://gdsc-practice-gcp/train /content/
if not os.path.exists('/content/test_val'):
    !gsutil -m cp -r gs://gdsc-practice-gcp/test_val /content/

Streaming output truncated to the last 5000 lines.
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20110831-001.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20110904-015.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20110906-009.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20110920-060.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20110921-019.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20110930-018.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20110930-021.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20120430-003.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20120430-030.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20120430-364.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20120503-039.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20120513-165.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20120513-178.jpg...
Copying gs://gdsc-practice-gcp/train/-1/PM-WWA-20120513-227.jpg...
Copying gs:

In [0]:
if not os.path.exists('/content/siamese_triplet_loss_custom'):
    !gsutil -m cp -r gs://gdsc-practice-gcp/siamese_triplet_loss_custom /content/

Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/inception_resnet/24-03-20-06-25-15/checkpoints/checkpoint0001.h5...
Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/inception_resnet/24-03-20-06-25-15/checkpoints/checkpoint0002.h5...
Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/inception_resnet/24-03-20-06-25-15/checkpoints/checkpoint0005.h5...
Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/inception_resnet/24-03-20-06-25-15/checkpoints/checkpoint0003.h5...
Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/inception_resnet/24-03-20-06-25-15/checkpoints/checkpoint0004.h5...
Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/inception_resnet/24-03-20-06-25-15/siamese_triplet_loss_custom_mobilenet_v2.h5...
Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/inception_resnet/24-03-20-06-25-15/siamese_triplet_loss_custom_mobilenet_v2_weights.h5...
Copying gs://gdsc-practice-gcp/siamese_triplet_loss_custom/siamese_triplet_

In [0]:
import os
import numpy as np
from pylab import *
#np.random.seed(0)
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import (Conv2D, ZeroPadding2D, Activation, Input, 
                                     concatenate, BatchNormalization, MaxPooling2D, 
                                     Concatenate, Lambda, Flatten, Dense, Reshape, 
                                     Layer)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform,he_uniform
from tensorflow.keras.preprocessing.image import load_img, array_to_img, img_to_array, ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model,normalize
from pathlib import Path
from sklearn.metrics import roc_curve,roc_auc_score
from math import floor
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import tensorflow as tf
import random
import gc
from datetime import datetime

In [0]:
def DrawPics(tensor,nb=0,template='{}',classnumber=None):
    if (nb==0):
        N = tensor.shape[0]
    else:
        N = min(nb,tensor.shape[0])
    fig=plt.figure(figsize=(16,2))
    nbligne = floor(N/20)+1
    for m in range(N):
        subplot = fig.add_subplot(nbligne,min(N,20),m+1)
        axis("off")
        plt.imshow(load_img(tensor[m]))
        if (classnumber!=None):
            subplot.title.set_text((template.format(classnumber[m])))

In [0]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
train_data_path = '/content/train'
import random 
id_folders = list(Path(train_data_path).glob("*"))
id_to_images = {}
image_to_id = {}
for folder in id_folders:
    if folder.name == "-1":
        continue
    pics = list(folder.glob("*.jpg"))
    if len(pics) == 1:
        continue
    id_to_images[folder.name] = list(pics)
    for p in pics:
        image_to_id[p] = folder.name


In [0]:
class_indices = list(id_to_images.keys())
img_indices = list(image_to_id.keys())
len(class_indices), len(img_indices)

(577, 2296)

In [0]:
batch_size = 32
batch_class_indices = np.random.choice(class_indices, size = batch_size, replace = False)
train_images = None
train_labels = []
for i, class_index in enumerate(batch_class_indices):
    print(class_index, i)
    imgs = np.random.choice(id_to_images[class_index], size = 2, replace = False)
    if i == 0:
        train_images = imgs
    else:
        train_images = np.hstack((train_images, imgs))
    train_labels.extend([class_index]*2)
len(train_labels), train_images.shape

5353 0
5420 1
3738 2
3895 3
3103 4
2882 5
2946 6
2624 7
3163 8
3821 9
5175 10
2778 11
3051 12
2894 13
3467 14
3473 15
3390 16
1078 17
2930 18
2327 19
3950 20
3716 21
5505 22
3186 23
3717 24
3894 25
3536 26
5258 27
5745 28
3180 29
5335 30
1371 31


(64, (64,))

In [0]:
for i in range(0, len(train_images), 2):
    DrawPics(train_images[i:i+2], nb = 2, template = 'train {}', classnumber = train_labels[i:i+2])

In [0]:
triplets = []
anchor_triplets = {}
for class_id in id_to_images.keys():
    #print(class_id)
    for i, img in enumerate(id_to_images[class_id]):
        anchor = [img]
        positives = id_to_images[class_id][:i] + id_to_images[class_id][i+1:]
        if len(positives) > 3:
            positives = np.random.choice(positives, size = 3, replace = False)
        negatives = []
        for class_id2 in id_to_images.keys():
            if class_id == class_id2:
                pass
            else:
                negatives.extend(id_to_images[class_id2])
        negatives = np.random.choice(negatives, size = 6, replace = False)
        anchor_triplets[img] = np.array(np.meshgrid(anchor, positives, negatives)).T.reshape(-1,3)
        triplets.append(anchor_triplets[img])
        
triplets = np.vstack(triplets)

In [0]:
val_triplet_indices = np.random.choice(range(len(triplets)), size = 2000, replace = False)
train_triplet_indices = [index for index in range(len(triplets)) if index not in val_triplet_indices]
val_triplets = triplets[val_triplet_indices]
train_triplets = triplets[train_triplet_indices]
len(train_triplets), len(val_triplets)

(30502, 2000)

In [0]:
import random
class DataGeneratorFromTriplets(Sequence):
    """
    Takes a list of triplets and precomputed embeddings and labels
    and feeds them in batches to a deep learning model.
    """

    def __init__(
        self,
        triplets,
        image_to_id, 
        batch_size=32,
        dim=(IMG_HEIGHT, IMG_WIDTH),
        shuffle=True,
    ):
        self.dim = dim
        self.batch_size = batch_size
        self.triplets = triplets
        self.image_to_id = image_to_id
        self.shuffle = shuffle
        self.indices = np.arange(len(self.triplets))
        self.img_gen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            brightness_range=[0.4, 1.0]
            #zoom_range=[0.7, 1.3],
        )
        self.on_epoch_end()

    def __len__(self):
        """
        Denotes the number of batches per epoch
        :return:
        """
        return int(np.floor(len(self.triplets) / self.batch_size))

    def __getitem__(self, index):
        """
        Generate one batch of data
        :param index:
        :return:
        """
        # Generate indexes of the batch
        indices = self.indices[
            index * self.batch_size : (index + 1) * self.batch_size
        ]
        # Find list of IDs
        triplets_temp = [self.triplets[k] for k in indices]
        # Generate data
        x, y = self.__data_generation(triplets_temp)
        return x, y

    def get_sample(self, index):
        """
        Gets one sample batch
        :param index:
        :return:
        """
        x, y = self.__getitem__(index)
        return x, y

    def on_epoch_end(self):
        """
        Updates indexes after each epoch
        :return:
        """
        if self.shuffle:
            np.random.shuffle(self.indices)
            np.random.shuffle(self.indices)
            np.random.shuffle(self.triplets)

    def __data_generation(self, triplets_temp):
        """
        Generates data containing batch_size samples
        :param triplets_temp:
        :return:
        """
        """# Initialization
        x = [np.empty((self.batch_size, *self.dim, 3)) for i in range(3)]
        y = [np.empty(self.batch_size) for i in range(2)]
        for i, pair in enumerate(triplets_temp):
            x[0][i,] = img_to_array(load_img(pair[0], target_size = self.dim))
            x[1][i,] = img_to_array(load_img(pair[1], target_size = self.dim))
            x[2][i,] = img_to_array(load_img(pair[2], target_size = self.dim))
            #y[0] = self.image_to_id[pair[0]]
            #y[1] = self.image_to_id[pair[1]]
            y[0][i, ] = 0
            y[1][i, ] = 0
        return x, y
        """
        x = [np.empty((self.batch_size, *self.dim, 3)) for i in range(3)]
        y = [np.empty(self.batch_size) for i in range(2)]
        for i, triplet in enumerate(triplets_temp):
            if i%2 == 0:
                random.shuffle(triplet[1:])
            img = img_to_array(load_img(triplet[0], target_size=self.dim))
            trans_args = self.img_gen.get_random_transform(self.dim)
            img = self.img_gen.apply_transform(img, trans_args)
            x[0][i,] = preprocess_input(img)
            
            img = img_to_array(load_img(triplet[1], target_size=self.dim))
            trans_args = self.img_gen.get_random_transform(self.dim)
            img = self.img_gen.apply_transform(img, trans_args)
            x[1][i,] = preprocess_input(img)
            
            img = img_to_array(load_img(triplet[2], target_size=self.dim))
            trans_args = self.img_gen.get_random_transform(self.dim)
            img = self.img_gen.apply_transform(img, trans_args)
            x[2][i,] = preprocess_input(img)
            
            if self.image_to_id[triplet[0]] == self.image_to_id[triplet[1]]:
                y[0][i, ] = 0
            else:
                y[0][i, ] = 1
            if self.image_to_id[triplet[0]] == self.image_to_id[triplet[2]]:
                y[1][i, ] = 0
            else:
                y[1][i, ] = 1
            
        return x, y


In [0]:
train_data_generator_from_triplets = DataGeneratorFromTriplets(train_triplets, image_to_id, batch_size = 32)
val_data_generator_from_triplets = DataGeneratorFromTriplets(val_triplets, image_to_id, batch_size = 32)

In [0]:
sample_data = train_data_generator_from_triplets.get_sample(103)
len(sample_data[0]), len(sample_data[1])

(3, 2)

In [0]:
sample_data[1]

[array([0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 0.]),
 array([1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0., 1.])]

In [0]:
def drawTriplets(tripletdata, nbmax=None):
    """display the three images for each triplets in the batch
    """
    tripletbatch = tripletdata[0]
    tripletlabels = tripletdata[1]
    labels = ["Anchor", "Positive", "Negative"]

    if (nbmax==None):
        nbrows = tripletbatch[0].shape[0]
    else:
        nbrows = min(nbmax,tripletbatch[0].shape[0])
                 
    for row in range(nbrows):
        fig=plt.figure(figsize=(16,2))
    
        for i in range(3):
            subplot = fig.add_subplot(1,3,i+1)
            axis("off")
            plt.imshow(array_to_img(tripletbatch[i][row,:,:,:]))
            if i == 0:
                label = 'Anchor'
                subplot.title.set_text(label)
            else:
                if tripletlabels[i-1][row] == 0:
                    label = 'Positive'
                else:
                    label = 'Negative'
                subplot.title.set_text(label)


In [0]:
drawTriplets(sample_data)

In [0]:
mobilenet = MobileNet(weights = 'imagenet', pooling = 'avg', input_shape = (IMG_HEIGHT, IMG_WIDTH, 3), include_top = False)
mobilenet.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
17227776/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None

In [0]:
inception_resnet = InceptionResNetV2(weights = 'imagenet', pooling = 'avg', input_shape = (IMG_HEIGHT, IMG_WIDTH, 3), include_top = False)
inception_resnet.summary()

219062272/219055592 [==============================] - 4s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_____________

In [0]:
EMBED_LENGTH = 1024 #mobilenet
#EMBED_LENGTH = 1536 #inception_resnet
def siamese_net_from_embeddings(learning_rate):    
    """
    Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    """
    #input_shape = [EMBED_LENGTH]
    embeddings_1 = Input(shape = EMBED_LENGTH)
    embeddings_2 = Input(shape = EMBED_LENGTH)

    mid = 32
    # Add a customized layer
    x1 = Lambda(lambda x: x[0] * x[1], name = 'multiply')([embeddings_1, embeddings_2])
    x2 = Lambda(lambda x: x[0] + x[1], name = 'add')([embeddings_1, embeddings_2])
    x3 = Lambda(lambda x: K.abs(x[0] - x[1]), name = 'abs_diff')([embeddings_1, embeddings_2])
    x4 = Lambda(lambda x: K.square(x), name = 'sqr_abs_diff')(x3)
    x = Concatenate()([x1, x2, x3, x4])
    x = Reshape((4, EMBED_LENGTH, 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x = Reshape((EMBED_LENGTH, mid, 1))(x)
    x = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x = Flatten(name='flatten')(x)
    
    # Weighted sum implemented as a Dense layer.
    similarity_score = Dense(1, use_bias=True, activation='sigmoid', name='output')(x)
    siamese_net = Model(inputs=[embeddings_1, embeddings_2], outputs=similarity_score)
    """
    #input_shape = [EMBED_LENGTH]
    embeddings_1 = Input(shape = EMBED_LENGTH)
    embeddings_2 = Input(shape = EMBED_LENGTH)

    x = Lambda(lambda x: K.abs(x[0] - x[1]), name = 'abs_diff')([embeddings_1, embeddings_2])

    similarity_score = Dense(1, use_bias=True, activation='sigmoid', name='output')(x)

    siamese_net = Model(inputs=[embeddings_1, embeddings_2], outputs=similarity_score)
    
    optimizer = Adam(learning_rate)
    siamese_net.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['mse', 'accuracy'])
    return siamese_net, optimizer

learning_rate = 64e-5
head_model, adam_opt = siamese_net_from_embeddings(learning_rate)
head_model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1024)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1024)]       0                                            
__________________________________________________________________________________________________
abs_diff (Lambda)               (None, 1024)         0           input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________


In [0]:
"""
input_anchor = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))
input_positive = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))
input_negative = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))

anchor_embedding = mobilenet(input_anchor )
positive_embedding = mobilenet(input_positive )
negative_embedding = mobilenet(input_negative )

anchor_l2_embed = Lambda(lambda x:K.l2_normalize(x, axis = -1))(anchor_embedding)
positive_l2_embed = Lambda(lambda x:K.l2_normalize(x, axis = -1))(positive_embedding)
negative_l2_embed = Lambda(lambda x:K.l2_normalize(x, axis = -1))(negative_embedding)

anchor_pos_dist = Lambda( lambda x: K.expand_dims(K.mean(K.square(x[0]-x[1]), axis = -1)))([anchor_l2_embed, positive_l2_embed])
anchor_neg_dist = Lambda( lambda x: K.expand_dims(K.mean(K.square(x[0]-x[1]), axis = -1)))([anchor_l2_embed, negative_l2_embed])

model = Model(inputs = [input_anchor, input_positive, input_negative ], outputs = [anchor_pos_dist, anchor_neg_dist])

model.summary()
"""
input_anchor = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))
input_positive = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))
input_negative = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3))

anchor_embedding = mobilenet(input_anchor)
positive_embedding = mobilenet(input_positive)
negative_embedding = mobilenet(input_negative)

#anchor_embedding = inception_resnet(input_anchor)
#positive_embedding = inception_resnet(input_positive)
#negative_embedding = inception_resnet(input_negative)

anchor_pos_similarity = head_model([anchor_embedding, positive_embedding])
anchor_neg_similarity = head_model([anchor_embedding, negative_embedding])

siamese_net = Model(inputs = [input_anchor, input_positive, input_negative], outputs = [anchor_pos_similarity, anchor_neg_similarity])
siamese_net.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
mobilenet_1.00_224 (Model)      (None, 1024)         3228864     input_5[0][0]                    
                                                                 input_6[0][0]              

In [0]:
gc.collect()
    
now = datetime.now().strftime("%d-%m-%y-%H-%M-%S")
model_folder = os.path.join('/content/', 'siamese_triplet_loss_custom', 'mobilenet', now)

if not os.path.exists(model_folder):
    os.makedirs(model_folder)

# Create a callback that saves the model's weights
checkpoints_dir = os.path.join(model_folder, "checkpoints",)

if os.path.exists(checkpoints_dir):
    pass
else:
    os.makedirs(checkpoints_dir)

# Ugly, but we need to have the placeholder within the folder.
checkpoints_dir = "%s/checkpoint{epoch:04d}.h5" % checkpoints_dir

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoints_dir, save_weights_only=True, verbose=1, period = 1
)


In [0]:
siamese_net.compile(optimizer = adam_opt, loss = 'binary_crossentropy', metrics = ['mse', 'acc'])
siamese_net.fit_generator(train_data_generator_from_triplets, epochs = 10, 
                          validation_data=val_data_generator_from_triplets, 
                          callbacks = [checkpoint_callback])

Epoch 1/10
  83/1270 [>.............................] - ETA: 52:33 - loss: 0.4794 - model_loss: 0.2398 - model_1_loss: 0.2395 - model_mean_squared_error: 0.0687 - model_acc: 0.9086 - model_1_mean_squared_error: 0.0746 - model_1_acc: 0.8926
Epoch 00001: saving model to /content/siamese_triplet_loss_custom/mobilenet/25-03-20-08-38-43/checkpoints/checkpoint0001.h5
1270/1270 [==============================] - 3639s 3s/step - loss: 0.7816 - model_loss: 0.3848 - model_1_loss: 0.3968 - model_mean_squared_error: 0.1202 - model_acc: 0.8350 - model_1_mean_squared_error: 0.1317 - model_1_acc: 0.8037 - val_loss: 0.4794 - val_model_loss: 0.2398 - val_model_1_loss: 0.2395 - val_model_mean_squared_error: 0.0687 - val_model_acc: 0.9086 - val_model_1_mean_squared_error: 0.0746 - val_model_1_acc: 0.8926
Epoch 2/10
  83/1270 [>.............................] - ETA: 52:36 - loss: 0.3326 - model_loss: 0.1747 - model_1_loss: 0.1579 - model_mean_squared_error: 0.0504 - model_acc: 0.9347 - model_1_mean_squared

In [0]:
siamese_net.save(os.path.join(model_folder, 'siamese_triplet_loss_custom_model.h5')
siamese_net.save_weights(os.path.join(model_folder, 'siamese_triplet_loss_custom_model_weights.h5')

In [0]:
!gsutil -m cp -r /content/siamese_triplet_loss_custom gs://gdsc-practice-gcp/

#!gsutil -m cp -r gs://gdsc-practice-gcp/siamese_triplet_loss_custom/siamese_triplet_loss_custom_model.h5 /content/
#!gsutil -m cp -r gs://gdsc-practice-gcp/siamese_triplet_loss_custom/siamese_triplet_loss_custom_model_weights.h5 /content/

In [0]:
#siamese_net = load_model('/content/siamese_triplet_loss_custom_model.h5')
#siamese_net.load_weights('/content/siamese_triplet_loss_custom_model_weights.h5')

siamese_net.compile(optimizer = adam_opt, loss = 'binary_crossentropy', metrics = ['mse', 'acc'])
siamese_net.fit_generator(train_data_generator_from_triplets, epochs = 10, 
                          validation_data=val_data_generator_from_triplets, 
                          callbacks = [checkpoint_callback], workers = 10)

In [0]:
#siamese_net.save(os.path.join(model_folder, 'siamese_triplet_loss_custom_model_v1.h5')
siamese_net.save_weights(os.path.join(model_folder, 'siamese_triplet_loss_custom_model_weights_v1.h5')

In [0]:
!gsutil -m cp -r /content/siamese_triplet_loss_custom gs://gdsc-practice-gcp/

#!gsutil -m cp -r gs://gdsc-practice-gcp/siamese_triplet_loss_custom/siamese_triplet_loss_custom_model.h5 /content/
#!gsutil -m cp -r gs://gdsc-practice-gcp/siamese_triplet_loss_custom/siamese_triplet_loss_custom_model_weights.h5 /content/

In [0]:
cnn = siamese_net.layers[3]
cnn.summary()

In [0]:
discriminator = siamese_net.layers[4]
discriminator.summary()